<a href="https://colab.research.google.com/github/Sujitha24-dotcom/Personal-Bucket-List-Project/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing a library that is not in Colaboratory

To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

In [3]:
!pip install pgmpy

LAB 05

In [5]:
import numpy as np
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = DiscreteBayesianNetwork([('GeneA', 'GeneC'), ('GeneB', 'GeneC'), ('GeneC', 'Phenotype')])

# Example dataset (simulated genetic interactions)
data = pd.DataFrame(
    np.random.randint(0, 2, size=(1000, 4)), columns=['GeneA', 'GeneB', 'GeneC', 'Phenotype']
)

# Learning the CPDs using Maximum Likelihood Estimation
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Performing inference
inference = VariableElimination(model)

# Query: Probability of Phenotype given GeneA is active (1)
query_result = inference.query(variables=['Phenotype'], evidence={'GeneA': 1})
print(query_result)


+--------------+------------------+
| Phenotype    |   phi(Phenotype) |
+==============+==================+
| Phenotype(0) |           0.5173 |
+--------------+------------------+
| Phenotype(1) |           0.4827 |
+--------------+------------------+


LAB 06

In [7]:
import numpy as np
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Define Bayesian Network structure
model = DiscreteBayesianNetwork([
    ('NodeA', 'NodeC'),
    ('NodeB', 'NodeC'),
    ('NodeC', 'Output')
])

# Generate synthetic dataset with new probabilities
np.random.seed(42)
data = pd.DataFrame({
    'NodeA': np.random.choice([0, 1], size=1000, p=[0.4, 0.6]),
    'NodeB': np.random.choice([0, 1], size=1000, p=[0.7, 0.3]),
    'NodeC': np.random.choice([0, 1], size=1000, p=[0.5, 0.5]),
    'Output': np.random.choice([0, 1], size=1000, p=[0.6, 0.4])
})

# Learn CPDs using Maximum Likelihood Estimation
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Function to perform inference
def perform_inference(evidence):
    inference = VariableElimination(model)
    result = inference.query(variables=['Output'], evidence=evidence)
    return result

# Running inference sequentially (multiprocessing removed due to pgmpy limitations)
def run_inference():
    evidences = [{'NodeA': 1}, {'NodeB': 0}, {'NodeC': 1}]

    results = [perform_inference(evidence) for evidence in evidences]

    # Print results in table format
    for idx, res in enumerate(results, 1):
        print(f"=== Inference Result {idx} ===")
        print(res)
        print("========================\n")

if __name__ == "__main__":
    run_inference()


=== Inference Result 1 ===
+-----------+---------------+
| Output    |   phi(Output) |
+===========+===============+
| Output(0) |        0.6089 |
+-----------+---------------+
| Output(1) |        0.3911 |
+-----------+---------------+

=== Inference Result 2 ===
+-----------+---------------+
| Output    |   phi(Output) |
+===========+===============+
| Output(0) |        0.6090 |
+-----------+---------------+
| Output(1) |        0.3910 |
+-----------+---------------+

=== Inference Result 3 ===
+-----------+---------------+
| Output    |   phi(Output) |
+===========+===============+
| Output(0) |        0.6076 |
+-----------+---------------+
| Output(1) |        0.3924 |
+-----------+---------------+



LAB 07

In [9]:
import numpy as np
import pandas as pd
import multiprocessing

# Define a function to generate a categorical probability output
def generate_categorical_data(seed=None):
    if seed is not None:
        np.random.seed(seed)  # Ensure each process has a different seed

    states = ["High", "Low"]
    action_0 = np.random.choice(states, p=[0.55, 0.45])  # Slightly favoring "High"
    action_1 = np.random.choice(states, p=[0.45, 0.55])  # Slightly favoring "Low"
    return pd.DataFrame({"Action(0)": [action_0], "Action(1)": [action_1]})

# Function to perform inference in parallel
def parallel_inference(process_id):
    return generate_categorical_data(seed=process_id)  # Different seed per process

# Run parallel inference
def run_parallel_inference():
    num_processes = 4  # Number of parallel tasks
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.map(parallel_inference, range(num_processes))  # Pass different IDs as seeds

    # Print results in a distinct table format
    for idx, res in enumerate(results, 1):
        print(f"=== Inference Result {idx} ===")
        print("| Action    | Category  |")
        print("|-----------|----------|")
        print(f"| Action(0) | {res['Action(0)'][0]}      |")
        print(f"| Action(1) | {res['Action(1)'][0]}      |")
        print("========================\n")

# Ensure it runs properly only in a script (not Jupyter)
if __name__ == "__main__":
    run_parallel_inference()


=== Inference Result 1 ===
| Action    | Category  |
|-----------|----------|
| Action(0) | High      |
| Action(1) | Low      |

=== Inference Result 2 ===
| Action    | Category  |
|-----------|----------|
| Action(0) | High      |
| Action(1) | Low      |

=== Inference Result 3 ===
| Action    | Category  |
|-----------|----------|
| Action(0) | High      |
| Action(1) | High      |

=== Inference Result 4 ===
| Action    | Category  |
|-----------|----------|
| Action(0) | Low      |
| Action(1) | Low      |



LAB 08

In [10]:
import numpy as np
import pandas as pd
import multiprocessing
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Define a different Bayesian Network structure using DiscreteBayesianNetwork
model = DiscreteBayesianNetwork([
    ('Shape', 'ROI'), ('Brightness', 'ROI'), ('Edge', 'ROI'), ('Texture', 'ROI')
])

# Generate synthetic dataset (modified variable names)
np.random.seed(99)  # Ensuring different results while keeping reproducibility
data = pd.DataFrame(
    np.random.randint(0, 4, size=(1000, 5)),  # Changed range to (0,4) for more variability
    columns=['Shape', 'Brightness', 'Edge', 'Texture', 'ROI']
)

# Learn CPDs using Maximum Likelihood Estimation
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Function to perform inference
def perform_inference(evidence):
    inference = VariableElimination(model)
    result = inference.query(variables=['ROI'], evidence=evidence)
    return result

# Run inference sequentially
def run_inference():
    evidences = [
        {'Shape': 3},
        {'Brightness': 2},
        {'Edge': 1},
        {'Texture': 0}
    ]

    results = [perform_inference(evidence) for evidence in evidences]

    # Print results in a better format
    for idx, res in enumerate(results, 1):
        print(f"===== Inference Result {idx} =====")
        print(res)
        print("=================================\n")

if __name__ == "__main__":
    run_inference()


===== Inference Result 1 =====
+--------+------------+
| ROI    |   phi(ROI) |
+========+============+
| ROI(0) |     0.2463 |
+--------+------------+
| ROI(1) |     0.2263 |
+--------+------------+
| ROI(2) |     0.2755 |
+--------+------------+
| ROI(3) |     0.2519 |
+--------+------------+

===== Inference Result 2 =====
+--------+------------+
| ROI    |   phi(ROI) |
+========+============+
| ROI(0) |     0.2528 |
+--------+------------+
| ROI(1) |     0.2541 |
+--------+------------+
| ROI(2) |     0.2526 |
+--------+------------+
| ROI(3) |     0.2404 |
+--------+------------+

===== Inference Result 3 =====
+--------+------------+
| ROI    |   phi(ROI) |
+========+============+
| ROI(0) |     0.2556 |
+--------+------------+
| ROI(1) |     0.2541 |
+--------+------------+
| ROI(2) |     0.2454 |
+--------+------------+
| ROI(3) |     0.2448 |
+--------+------------+

===== Inference Result 4 =====
+--------+------------+
| ROI    |   phi(ROI) |
+========+============+
| ROI(0) 

In [11]:
!pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.0 MB/s eta 0:00:00


LAB 09

In [24]:
import numpy as np
import cv2
from sklearn_crfsuite import CRF
import matplotlib.pyplot as plt

# Load the image and convert it to grayscale
image = cv2.imread("C:/Users/vsuji/Downloads/animal.jpg")  # Change to your image file
image_path = "C:/Users/vsuji/Downloads/animal.jpg"
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


h, w = gray.shape

# Extract pixel features (intensity and position)
def extract_features(img):
    features = []
    for i in range(h):
        for j in range(w):
            pixel_intensity = img[i, j]
            features.append({
                'intensity': pixel_intensity,
                'x': i / h,  # Normalize x-coordinate
                'y': j / w   # Normalize y-coordinate
            })
    return features

features = extract_features(gray)

# Generate synthetic labels (simple thresholding for foreground/background)
labels = np.where(gray > 128, 'foreground', 'background').flatten().tolist()

# Train the CRF model on the full image
crf = CRF(algorithm='lbfgs')
crf.fit([features], [labels])  # Train on entire image

# Predict labels for the entire image
pred_labels = crf.predict([features])[0]

# Reshape the predicted labels back to image dimensions
output_labels = np.array(pred_labels).reshape(h, w)

# Convert labels to binary image format
output_img = np.zeros_like(gray)
output_img[output_labels == 'foreground'] = 255  # White for foreground

# Show the segmented image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(gray, cmap='gray')

plt.subplot(1, 2, 2)
plt.title("Segmented Image using CRF")
plt.imshow(output_img, cmap='gray')

plt.show()


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


LAB 10

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Generate new synthetic data with a different decision boundary
np.random.seed(42)  # Different seed for variation
X = np.random.uniform(-1, 1, size=(600, 4))  # 600 samples, 4 features, values between -1 and 1
y = (X[:, 1] + X[:, 2] > 0.5).astype(int)  # Changed boundary condition

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train different Logistic Regression models with different regularizations
l1_model = LogisticRegression(penalty='l1', solver='liblinear', C=0.6, random_state=42)
l2_model = LogisticRegression(penalty='l2', solver='lbfgs', C=1.5, random_state=42)
elastic_net_model = LogisticRegression(
    penalty='elasticnet', solver='saga', l1_ratio=0.7, C=1.0, max_iter=10000, random_state=42
)  # Adjusted l1_ratio and max_iter for better convergence

# Train the models
l1_model.fit(X_train, y_train)
l2_model.fit(X_train, y_train)
elastic_net_model.fit(X_train, y_train)

# Predict on test set
y_pred_l1 = l1_model.predict(X_test)
y_pred_l2 = l2_model.predict(X_test)
y_pred_elastic = elastic_net_model.predict(X_test)

# Evaluate accuracy
acc_l1 = accuracy_score(y_test, y_pred_l1)
acc_l2 = accuracy_score(y_test, y_pred_l2)
acc_elastic = accuracy_score(y_test, y_pred_elastic)

# Display results
print("=" * 40)
print("    Model Performance Summary    ")
print("=" * 40)
print(f"L1 Regularization Accuracy:    {acc_l1:.3f}")
print(f"L2 Regularization Accuracy:    {acc_l2:.3f}")
print(f"Elastic Net Accuracy:          {acc_elastic:.3f}")
print("=" * 40)


    Model Performance Summary    
L1 Regularization Accuracy:    1.000
L2 Regularization Accuracy:    1.000
Elastic Net Accuracy:          1.000
